In [ ]:
#da eseguire solo la prima volta per installare la libreria
!pip install pathlib

In [2]:
#libraries
from geopy.geocoders import Yandex
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date as dt
from tqdm import tqdm
from pathlib import Path
import time

i = 700
pathlist = Path('').glob('**/*gps.csv')

for path in pathlist:
    
    i += 1
    
    #import files
    file = str(path)
    print(file, i)
    h = pd.read_csv(file, skiprows=1) #gps
    r = pd.read_csv(str(i)+'_sj.csv') #science_journal
    
    #order/filter column
    col = []
    for i in h.columns:
        i = i.replace(' ', '')
        col.append(i)
    h.columns = col
    h = h.filter(['Time','Latitude','Longitude','Elevation','Speed'])
    
    #to datetime
    h['Time'] = '2019-04-25 '+ h['Time']
    h['Time'] = pd.to_datetime(h.Time, format='%Y-%m-%d %H:%M:%S')
    
    #adjust Speed
    h['Speed'] = h.Speed*4
    
    #put n. person, type of car
    h['N_person'] = int(file.split('_')[2])
    h['Car_type'] = int(file.split('_')[3])
    
    #create diff
    data1=h.iloc[0]['Time']
    h['Diff']=(h['Time']-data1).dt.seconds
    h['Diff']=h['Diff']*1000
    h = h.drop_duplicates(subset=['Diff'], keep='first')
    
    #assign a reference to sj_file
    l=[str(i) for i in h.index]
    gap=[h.iloc[i]['Diff'] for i in range(len(h))]
    del l[-1]
    r=r.assign(tag=pd.cut(r['relative_time'],gap,labels=l))#,duplicates='drop'))
    r['tag']=r['tag'].fillna(value='0')
    
    #adjust sj
    sj_new=r.filter(['relative_time','AccZ','tag'])
    sj_new.columns=['Time','Acc','Reference']
    sj_new['Reference'] = pd.to_numeric(sj_new['Reference'])
    
    #merged df
    df=pd.merge(sj_new,h,left_on='Reference',right_on=h.index)
    df=df.drop(columns=['Reference','Diff'])
    df.columns=['Elapse(ms)','Acc','Date','Latitude','Longitude','Elevation','Speed(Km/h)','N_person','Car_type']
    df=df.reindex(['Date','Elapse(ms)','Latitude','Longitude','Elevation','Speed(Km/h)','Acc','N_person','Car_type'],axis=1)              
    df=df[df['Speed(Km/h)']>1]
    
    #reverse geopy
    #ALE INSERISCI CODICE QUI!
    
    #save file
    df.to_csv(str(i)+'_Record.csv', index=False)